# Basic RNA circuit simulation 



In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
from bioreaction.simulation.basic_sim import convert_model

import jax
import numpy as np
import os
import sys

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-paper')
os.environ["TF_CPP_MIN_LOG_LOVEL"] = "0"
jax.config.update('jax_platform_name', 'gpu')


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)


from src.utils.data.data_format_tools.common import load_json_as_dict
from src.utils.common.setup_new import prepare_config, construct_circuit_from_cfg
from src.utils.circuit.agnostic_circuits.circuit_manager_new import CircuitModeller
from tests_local.shared import five_circuits

config = load_json_as_dict('../tests_local/configs/simple_circuit.json')


In [72]:



def update_model_rates(model, a=None, d=None, ka=None, kd=None):
    for i, r in enumerate(model.reactions):
        if not r.input:  # 0 -> RNA
            if a is not None:
                model.reactions[i].forward_rate = a[model.species.index(
                    r.output[0])]
                model.reactions[i].reverse_rate = 0
        elif not r.output:  # RNA -> 0
            if d is not None:
                model.reactions[i].forward_rate = d[model.species.index(
                    r.input[0])]
                model.reactions[i].reverse_rate = 0
        else: # unbound RNA -> bound RNA
            if ka is not None:
                model.reactions[i].forward_rate = ka[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
            if kd is not None:
                model.reactions[i].reverse_rate = kd[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
    return model


def make_params(model, scale_rates=True):
    sim_model = convert_model(model)

    if scale_rates:
        m = np.max([sim_model.forward_rates.max(), sim_model.reverse_rates.max()])
    else:
        m = 1
    inputs = sim_model.inputs
    outputs = sim_model.outputs
    forward_rates = sim_model.forward_rates/m
    reverse_rates = sim_model.reverse_rates/m
    return inputs, outputs, forward_rates, reverse_rates, m

Use the 5 tester circuits :)

In [73]:
config = prepare_config(config)
config['include_prod_deg'] = True
config['simulation']['use_rate_scaling'] = False

q = 2
p = 1
config['molecular_params_factor'] = p
config['simulation']['interaction_factor'] = 1
config['molecular_params']['creation_rate'] = config['molecular_params']['creation_rate'] * q
config['molecular_params']['creation_rate' + '_per_molecule'] = config['molecular_params']['creation_rate' + '_per_molecule'] * q
# config['simulation']['dt'] = 0.1
# config['molecular_params']['creation_rate'] = config['molecular_params']['creation_rate'] * p
# config['molecular_params']['degradation_rate'] = config['molecular_params']['degradation_rate'] * p
circuits, config, result_writer = five_circuits(config)

experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/binding_rates_dissociation/0_weak_binding_rates_dissociation.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/eqconstants/0_weak_eqconstants.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/energies/0_weak_energies.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/binding_sites/0_weak_binding_sites.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/binding_rates_dissociation/1_med_weak_binding_rates_dissociation.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead

In [74]:




def load_circuit(top_dir, circuit_name, config):
    dp = os.path.join(top_dir, 'circuits', circuit_name + '.fasta')
    interactions = {'binding_rates_association': config['molecular_params']['association_binding_rate' + '_per_molecule'],
                    'binding_rates_dissociation': os.path.join(top_dir, 'binding_rates_dissociation', circuit_name + '_' + 'binding_rates_dissociation' + '.csv'),
                    'eqconstants': os.path.join(top_dir, 'eqconstants', circuit_name + '_' + 'eqconstants' + '.csv'),
                    'energies': os.path.join(top_dir, 'energies', circuit_name + '_' + 'energies' + '.csv'),
                    'binding_sites': os.path.join(top_dir, 'binding_sites', circuit_name + '_' + 'binding_sites' + '.csv')}
    
    return construct_circuit_from_cfg({
        'data_path': dp,
        'interactions': interactions
    }, config)


tester_circuit = load_circuit(top_dir='data/ensemble_mutation_effect_analysis/2023_04_04_092945/generate_species_templates',
             circuit_name='toy_mRNA_circuit_1844', config=config)

# config['interactions'] = {}
config['interactions'] = None
# tester_circuit = construct_circuit_from_cfg(
#     {'data_path': '../tester.fasta'}, config)
circuits.append(tester_circuit)


In [75]:

circuit_modeller = CircuitModeller(result_writer=result_writer, config=config)
circuits = circuit_modeller.batch_circuits(circuits=circuits, methods={"compute_interactions": {}})
i = -1
# for i in range(len(circuits)):
#     circuits[i].interactions.binding_rates_association = circuits[i].interactions.binding_rates_association * q
#     circuits[i].interactions.binding_rates_dissociation = circuits[i].interactions.binding_rates_dissociation * q
#     circuits[i].update_species_simulated_rates(circuits[i].interactions)


simulator.py:check_IntaRNA_path():115: Could not detect IntaRNA on system: CompletedProcess(args='which IntaRNA', returncode=1, stdout='', stderr='') WARNING
circuit_manager_new.py:batch_circuits():559: 	From 6 circuits, a total of 6 mutated circuits will be simulated. WARNING
circuit_manager_new.py:batch_circuits():566: 		Starting new round of viable circuits (0 - 6 / 6) WARNING
circuit_manager_new.py:batch_circuits():584: 		Making subcircuits 0 - 6 took 1.1e-05s WARNING
circuit_manager_new.py:batch_circuits():590: 	Batching 0 - 6 circuits (out of 0 - 6 (total: 6)) (Circuits: 0 - 6 of 6) WARNING
circuit_manager_new.py:run_batch():621: 		Running 6 Subcircuits - 0_weak: compute_interactions WARNING
circuit_manager_new.py:run_batch():657: 		Method compute_interactions took 0.024906s WARNING
circuit_manager_new.py:batch_circuits():604: Single batch: 0:00:00.025929 
Projected time: 0.025928999999999997s 
Total time: 0:00:00.025939 WARNING


In [76]:
circuits[i].interactions.binding_rates_dissociation

array([[0.45263996, 0.45263996, 0.45263996],
       [0.45263996, 0.45263996, 0.45263996],
       [0.45263996, 0.45263996, 0.45263996]])

In [77]:
circuits[i].interactions.binding_rates_association

array([[0.00301916, 0.00301916, 0.00301916],
       [0.00301916, 0.00301916, 0.00301916],
       [0.00301916, 0.00301916, 0.00301916]])

In [78]:
config['molecular_params']['creation_rate']

35.25

In [79]:
config['molecular_params']['degradation_rate']


0.17625

In [80]:
circuits = circuit_modeller.batch_circuits(
    circuits=circuits, 
    methods={
        # "compute_interactions": {},
        "init_circuits": {'batch': True},
        'simulate_signal_batch': {'ref_circuit': None,
                                  'batch': config['simulation']['use_batch_mutations']},
        'write_results': {'no_visualisations': False, # config['experiment']['no_visualisations'],
                          'no_numerical': False} #config['experiment']['no_numerical']}
    }
)

circuit_manager_new.py:batch_circuits():559: 	From 6 circuits, a total of 6 mutated circuits will be simulated. WARNING
circuit_manager_new.py:batch_circuits():566: 		Starting new round of viable circuits (0 - 6 / 6) WARNING
circuit_manager_new.py:batch_circuits():584: 		Making subcircuits 0 - 6 took 1.4e-05s WARNING
circuit_manager_new.py:batch_circuits():590: 	Batching 0 - 6 circuits (out of 0 - 6 (total: 6)) (Circuits: 0 - 6 of 6) WARNING
circuit_manager_new.py:run_batch():621: 		Running 6 Subcircuits - 0_weak: init_circuits WARNING
circuit_manager_new.py:run_batch():657: 		Method init_circuits took 1.462269s WARNING
circuit_manager_new.py:run_batch():621: 		Running 6 Subcircuits - 0_weak: simulate_signal_batch WARNING
circuit_manager_new.py:prepare_internal_funcs():489: Signal differs between circuits, but only first signal used for simulation. WARNING


Done:  0:00:01.397943


circuit_manager_new.py:simulate_signal_batch():337: 		Simulating signal took 1.959654s WARNING


Done:  0:00:01.939474


circuit_manager_new.py:simulate_signal_batch():395: 		Calculating analytics took 1.062458s WARNING
circuit_manager_new.py:run_batch():657: 		Method simulate_signal_batch took 3.121437s WARNING
circuit_manager_new.py:run_batch():621: 		Running 6 Subcircuits - 0_weak: write_results WARNING
circuit_manager_new.py:run_batch():657: 		Method write_results took 1.865931s WARNING
circuit_manager_new.py:batch_circuits():604: Single batch: 0:00:06.451751 
Projected time: 6.451750999999999s 
Total time: 0:00:06.451761 WARNING


Log

1.
q = 5
p = 2
-> not great

2. 
q = 2
p = 5
-> good, the weak ones could be more step-like

3. 
q = 3
p = 5
-> Same-looking slope for signal, but doesn't reach as high as 2. 

4. 
q = 2
p = 10
-> better, more step-like on strong, but still a bit dragging on weaker ones

5. 
q = 2
p = 15
-> even better, more step-like, but no overshoot like you get in the no prod/deg

6. 
q = 5
p = 50
-> spiky, but now a bit too dominating with prod / deg




